### Anotação das cadeias de correferências

In [ ]:
import pandas as pd
import pickle

PATH_REDACOES = "../ccscore/data/Corpus_Redacoes_Dissertacao.pickle"

In [ ]:
# Carrega a base de redações
df_redacao = pickle.load(open(PATH_REDACOES, 'rb'))
df_redacao

In [ ]:
# Conta da quantidade de arquivos de redações
total_redacoes = df_redacao.shape[0]
print(f"Total de redações: {total_redacoes}")

In [ ]:
import spacy

nlp = spacy.load('pt_core_news_lg')

In [ ]:
# Contagem de palavras das redacoes
total_palavras = 0

for i,redacao in df_redacao.iterrows():
    texto = redacao['Texto']
    texto = texto.replace("\n", ' ')

    doc = nlp(texto)
    total_palavras += len([token.text for token in doc if token.is_punct != True])

total_palavras

In [ ]:
print(f"Total de palavras de todas as redações: {total_palavras}")
print(f"Média de palavras por redação: {round(total_palavras/total_redacoes)}")

In [ ]:
#redacao_exemplo = nlp(df_redacao[["Texto"]][0])
i_redacao = 1691
print(df_redacao["Texto"][i_redacao])
df_redacao.iloc[i_redacao]

In [ ]:
# Grava os arquivos as redações em arquivos para anotação
# utilizando o software CORP
if False:
    for i,redacao in df_redacao.iterrows():
        texto = redacao['Texto']
        arq_redacao = f'../CORP/Entrada/redacao_{i}.txt'
        try:
            with open(arq_redacao, 'w') as f:
                f.write(texto)
        except IOError as e:
            print(f"Erro ao processar o arquivo {arq_redacao}")


In [ ]:
import pandas as pd

df = pd.DataFrame([[10,20,30],[11,22,33],[1,2,3]], 
                  columns=['col1', 'col2', 'col3'])

df['x'] = ''

#df['x'][0] = 'a'
df.at[0,'x'] = 'a'
df.at[1,'x'] = 'b'
df.at[2,'x'] = 'c'

df

In [ ]:
!pip install xmltodict

In [ ]:
# Carga das anotações das cadeias de coreferência
import glob
import xmltodict

PATH_CORREF='../CORP/Saida/XML'

cadeias_corref = {}
erros = []
for name in glob.glob(f'{PATH_CORREF}/*.xml'):    
    id_redacao = name.split('redacao_')[1]
    id_redacao = int(id_redacao.split('.')[0])
    #print(name)
    #print(id_redacao)
    texto_xml = ""
    try:
        with open(name, 'r', encoding='utf-8') as f:
            texto_xml = " ".join(f.readlines())
    except IOError:
        print("Erro na abertura do arquivo")
        
    try:
        cadeias_corref[id_redacao] = xmltodict.parse(texto_xml)    
    except Exception as e:
        erros.append((id_redacao, name))
        print("Erro:", str(e))
        
    
#corp_red['ConteudoXML']['Cadeias']['Cadeia_21']
#corp_red['ConteudoXML'].keys()

In [ ]:
erros


In [ ]:
PATH_CORP = "../ccscore/data/Corpus_Redacoes_CORP.pickle"

with open(PATH_CORP, 'wb') as f:    
    pickle.dump(cadeias_corref, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
num_sentenca = 3
cadeias_corref[0]['ConteudoXML']['Sentencas'][f"sentenca_{num_sentenca}"]['@conteudo']
cadeias_corref[0]['ConteudoXML']['Cadeias']

In [ ]:
def get_sentenca(num_redacao, num_sentenca):
    return cadeias_corref[num_redacao]['ConteudoXML']['Sentencas'][f"sentenca_{num_sentenca}"]['@conteudo']

In [ ]:
num_redacao = 0
cadeias = cadeias_corref[num_redacao]['ConteudoXML']['Cadeias'].items()
for cadeia, sns in cadeias:
    print(f"\n{cadeia}")
    for elem_cadeia in sns.items():
        for elem in elem_cadeia[1]:
            #print(elem)
            sn_elem = elem['@sintagma']
            categoria = elem['@Categoria']
            print(f"\t{categoria} - {sn_elem}")
            s = get_sentenca(num_redacao, elem['@sentenca'])
            s = s.replace(sn_elem, f"<<{sn_elem}>>")
            print(f"\t{s}")
            
            
        